___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg" width="350px" height="180px" />


# <font color= #8A0829> Data modelling lab </font>
- <Strong> Teacher: Sara Eugenia Rodríguez </Strong>
- <Strong> Year </Strong>: 2022
___

<p style="text-align:right;"> Image from: https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg</p>

### <font color= #2E9AFE> Classification with XGboost</font>


#### Characteristics
- Secuential set of trees that learn from previous trees 




#### Parameters

- learning_rate (eta)
- depth
- Gamma (para podar el árbol)
- Lambda (regularización)
- Min child weight
- scale_pos_weigth

#### Advantages

- Can handle big datasets
- can handle null values
- Precise results
- Fast

#### Disadvantages
- Uses a lot of resources
- Is convenient to transform all data to numerical

## Clasification

### Data

Data from:
https://www.kaggle.com/yeanzc/telco-customer-churn-ibm-dataset


In [1]:
#Libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
#Import data
data = pd.read_excel('Telco_customer_churn.xlsx')

In [3]:
#Snippet of the data
data.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [4]:
data.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

**Data preprocessing**

In [5]:
#Get rid off spaces in columns
data.columns = data.columns.str.replace(" ","_")


In [6]:
data.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip_Code',
       'Lat_Long', 'Latitude', 'Longitude', 'Gender', 'Senior_Citizen',
       'Partner', 'Dependents', 'Tenure_Months', 'Phone_Service',
       'Multiple_Lines', 'Internet_Service', 'Online_Security',
       'Online_Backup', 'Device_Protection', 'Tech_Support', 'Streaming_TV',
       'Streaming_Movies', 'Contract', 'Paperless_Billing', 'Payment_Method',
       'Monthly_Charges', 'Total_Charges', 'Churn_Label', 'Churn_Value',
       'Churn_Score', 'CLTV', 'Churn_Reason'],
      dtype='object')

In [7]:
#numerical field as object
data['Total_Charges'].unique()

array([108.15, 151.65, 820.5, ..., 7362.9, 346.45, 6844.5], dtype=object)

In [9]:
data.replace(" ",np.nan,inplace=True)

In [10]:
#Change to numerical

data['Total_Charges'] = pd.to_numeric(data['Total_Charges'])

In [11]:
data['Total_Charges'].unique()

array([ 108.15,  151.65,  820.5 , ..., 7362.9 ,  346.45, 6844.5 ])

In [12]:
#Dropping columns with y information
data.drop(['Churn_Label','Churn_Reason','Churn_Score','CLTV'], axis=1, inplace=True)

In [14]:
# Dropping variables with low variance and some other columns
data.nunique()

CustomerID           7043
Count                   1
Country                 1
State                   1
City                 1129
Zip_Code             1652
Lat_Long             1652
Latitude             1652
Longitude            1651
Gender                  2
Senior_Citizen          2
Partner                 2
Dependents              2
Tenure_Months          73
Phone_Service           2
Multiple_Lines          3
Internet_Service        3
Online_Security         3
Online_Backup           3
Device_Protection       3
Tech_Support            3
Streaming_TV            3
Streaming_Movies        3
Contract                3
Paperless_Billing       2
Payment_Method          4
Monthly_Charges      1585
Total_Charges        6530
Churn_Value             2
dtype: int64

In [15]:
#We drop CustomerID, Lat Long, count, country y state
data.drop(['CustomerID','Lat_Long','Count','Country','State'], axis=1, inplace=True)

In [17]:
data.head()

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Value
0,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
1,Los Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
2,Los Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1
3,Los Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
4,Los Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1


In [18]:
#Get rid off spaces in df
data.replace(" ","_", regex=True, inplace=True)

In [19]:
data.head()

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Value
0,Los_Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed_check,53.85,108.15,1
1,Los_Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic_check,70.70,151.65,1
2,Los_Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic_check,99.65,820.50,1
3,Los_Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic_check,104.80,3046.05,1
4,Los_Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank_transfer_(automatic),103.70,5036.30,1


In [20]:
#Split X and Y
X = data.drop('Churn_Value', axis=1).copy()
X.head(3)

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges
0,Los_Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed_check,53.85,108.15
1,Los_Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,No,No,No,No,No,Month-to-month,Yes,Electronic_check,70.70,151.65
2,Los_Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic_check,99.65,820.50


In [21]:
y = data['Churn_Value'].copy()

In [23]:
#Using one hot
newX = pd.get_dummies(X, columns=['City','Gender','Senior_Citizen','Partner','Dependents',
                           'Phone_Service','Multiple_Lines','Internet_Service','Online_Security',
                          'Online_Backup','Device_Protection','Tech_Support','Streaming_TV',
                          'Streaming_Movies','Contract','Paperless_Billing','Payment_Method'])
newX.head(3)

,Zip_Code,Latitude,Longitude,Tenure_Months,Monthly_Charges,Total_Charges,City_Acampo,City_Acton,City_Adelanto,City_Adin,...,Streaming_Movies_Yes,Contract_Month-to-month,Contract_One_year,Contract_Two_year,Paperless_Billing_No,Paperless_Billing_Yes,Payment_Method_Bank_transfer_(automatic),Payment_Method_Credit_card_(automatic),Payment_Method_Electronic_check,Payment_Method_Mailed_check
0,90003,33.964131,-118.272783,2,53.85,108.15,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,90005,34.059281,-118.307420,2,70.70,151.65,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,90006,34.048013,-118.293953,8,99.65,820.50,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0


In [24]:
#Checking balance
sum(y)/len(y)

#26% of all records are churn

0.2653698707936959

**Model**

In [25]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(newX, y, random_state=42, stratify=y) #stratify is used for classification models

In [26]:
sum(y_train)/len(y_train)

0.2654297614539947

In [27]:
sum(y_test)/len(y_test)

0.26519023282226006

In [29]:
#xgboost
modelo = xgb.XGBClassifier(objective='binary:logistic', missing=1,early_stopping_rounds=10, seed=42) #early_stopping_rounds: stop building models if score doesn't improve
#train model
modelo.fit(X_train, y_train, verbose=True,
          eval_set=[(X_test, y_test)])

[0]	validation_0-logloss:0.57536
[1]	validation_0-logloss:0.51107
[2]	validation_0-logloss:0.47265
[3]	validation_0-logloss:0.44765
[4]	validation_0-logloss:0.43323
[5]	validation_0-logloss:0.42193
[6]	validation_0-logloss:0.41643
[7]	validation_0-logloss:0.41259
[8]	validation_0-logloss:0.41057
[9]	validation_0-logloss:0.41045
[10]	validation_0-logloss:0.41026
[11]	validation_0-logloss:0.40891
[12]	validation_0-logloss:0.40901
[13]	validation_0-logloss:0.40930
[14]	validation_0-logloss:0.40864
[15]	validation_0-logloss:0.40808
[16]	validation_0-logloss:0.40770
[17]	validation_0-logloss:0.40783
[18]	validation_0-logloss:0.40854
[19]	validation_0-logloss:0.40847
[20]	validation_0-logloss:0.40881
[21]	validation_0-logloss:0.40942
[22]	validation_0-logloss:0.41032
[23]	validation_0-logloss:0.41112
[24]	validation_0-logloss:0.41159
[25]	validation_0-logloss:0.41199
[26]	validation_0-logloss:0.41410


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=1, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [37]:
# Test the performance
from sklearn.metrics import (accuracy_score,precision_score,recall_score)


yhat = modelo.predict(X_test)
accu = accuracy_score(y_test,yhat)
prec = precision_score(y_test,yhat)
reca = recall_score(y_test,yhat)
print('Accuracy\t Precision\t Recall\n %0.3f\t           %0.3f\t %0.3f'%(accu,prec,reca))

Accuracy	 Precision	 Recall
 0.800	           0.645	 0.544



*scale_pos_weight* will help with unbalanced data


#### Cross validation and GridSearch 

hiperparameters:
- max_depth 
- learning_rate (eta)
- gamma
- reg_lambda
- scale_pos_weigth

In [38]:
#because of time, we'll do 2 rounds of finetunning, we firstly run the commented params and then by logic make the other grid

#param_grid={
#    'max_depth':[3,4,5],
#    'learning_rate':[0.1,0.01,0.05],
#    'gamma':[0, 0.25,1],
#    'reg_lambda':[0,1,10],
#    'scale_pos_weight':[1,3,5]
#}

#Round 2
param_grid={
    'max_depth':[4],
    'gamma':[0.25],
    'reg_lambda':[10,20,100],
    'scale_pos_weight':[3]
}

optimal_params = GridSearchCV(
                            estimator=xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            subsample=0.9,
                            colsample_bytree=0.5),
                            param_grid=param_grid,
                            scoring='roc_auc',
                            verbose=0,
                            n_jobs=10,
                            cv=3)

optimal_params.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  verbose=False)
optimal_params.best_params_

{'gamma': 0.25, 'max_depth': 4, 'reg_lambda': 100, 'scale_pos_weight': 3}

{'gamma': 0.25,
 'learning_rate': 0.1,
 'max_depth': 4,
 'reg_lambda': 10,
 'scale_pos_weight': 3}

In [40]:
#Build model with optimal parameters
modelo = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                           gamma=0.25,
                           max_depth=4,
                           reg_lambda=10,
                           early_stopping_rounds=10,
                           scale_pos_weight=3,
                            subsample=0.9,
                            colsample_bytree=0.5)

modelo.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [45]:
# Model performance
from sklearn.metrics import (accuracy_score,precision_score,recall_score)


yhat = modelo.predict(X_test)
accu = accuracy_score(y_test,yhat)
prec = precision_score(y_test,yhat)
reca = recall_score(y_test,yhat)
print('Accuracy\t Precision\t Recall\n %0.3f\t           %0.3f\t  %0.3f'%(accu,prec,reca))

Accuracy	 Precision	 Recall
 0.755	           0.524	  0.831


#### References...
https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning